In [1]:
#Import libraries

import pandas as pd
import requests
import numpy as np
from datetime import date
import pyodbc

In [2]:
# API Key and Series ID from EIA

api_key = '131d8d818f34ec47f9335aea892e52d2'
seriesid = 'PET.RCLC1.D'

## Extract

In [3]:
#url = 'http://api.eia.gov/series/?api_key=' + api_key +'&series_id=' + PADD_KEY[i]
url = 'http://api.eia.gov/series/?api_key='+api_key+'&series_id='+seriesid
r = requests.get(url)
json_data = r.json()
    
if r.status_code == 200:
    print('Success!')
else:
    print('Error')

df = pd.DataFrame(json_data.get('series')[0].get('data'),
                 columns = ['Date', 'Dollars_per_Barrel'])

Success!


## Transform

In [4]:
# Create date as datetype datatype
df['Year'] = df['Date'].astype(str).str[:4]
df['Month'] = df['Date'].astype(str).str[4:6]
df['Day'] = df['Date'].astype(str).str[6:]
df['Date'] = pd.to_datetime(df[['Year','Month','Day']])

In [5]:
# Now dropping the unnecessary columns >>> 'Year','Month','Day'
df = df.drop(['Year','Month','Day'], axis=1) 

In [6]:
# # Set Date as the index column
# df.set_index('Date', drop=True, inplace=True)

## Load

In [8]:
# Set connection to the database 
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=LAPTOP-5BK23GEM;'
                      'Database=demodb;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()

In [9]:
# Push the data the CrudeOil_FutureContract table in database

for row in df.itertuples():
    cursor.execute('''
                INSERT INTO demodb.dbo.CrudeOil_FutureContract (Date, Dollars_per_Barrel)
                VALUES (?,?)
                ''',
                row.Date, 
                row.Dollars_per_Barrel
                )
conn.commit()